In [1]:
import pickle
import pandas as pd
import deepchem as dc
import torch
from torch_geometric.data import Data
from sklearn.model_selection import train_test_split

C:\Users\noton\miniconda3\envs\Deep2\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm



Instructions for updating:
experimental_relax_shapes is deprecated, use reduce_retracing instead


Skipped loading modules with pytorch-geometric dependency, missing a dependency. No module named 'dgl'
Skipped loading some Jax models, missing a dependency. No module named 'jax'


In [2]:
torch.manual_seed(0)
df = pd.read_csv('data_Random/data_Random.csv')
df

,SMILES,ABCGG,AATSC3d,ATSC3d,GGI10,ATSC4i,BCUTp-1l,Kier3,AATS8p,Kier2,fr_NH0,VSA_EState3,Kappa3,Kappa2,PEOE_VSA6,BertzCT,EState_VSA3
0,Cn1cccc1N1c2ccccc2C(C)(Cc2ccc(C(=O)C(F)(F)F)cc...,33.173429,-0.065944,-15.430914,1.021435,-33.215854,0.526872,8.192784,1.501815,17.762466,5,2.280330,6.097227,13.881270,97.039602,2944.571750,11.984273
1,c1cc(Pc2cccc(C3c4ccccc4-c4ccccc43)c2)cc(-c2ccc...,23.638334,-0.062054,-11.604089,0.269470,10.212686,1.008237,5.972299,1.561482,13.721707,1,5.295506,4.430623,10.737517,148.107140,2262.995876,8.580697
2,O=c1cc(-c2c(F)c(F)cc(F)c2F)[nH]c2ccc(-c3c(-c4c...,29.407498,-0.078344,-14.885309,0.502808,4.589952,0.543321,6.538906,1.736901,15.306926,3,4.985782,4.720717,11.689039,64.970755,3002.628357,5.563451
3,O=S1(=O)c2ccccc2-c2ccc(-c3cc4ccccc4c4ccccc34)cc21,17.289311,-0.110794,-12.408889,0.076325,-20.135406,0.761341,3.111111,1.650159,7.548817,0,4.645536,2.250816,5.786129,78.862772,1563.153065,9.790967
4,C=C[SH]1C=CC=C1c1cc(-c2ccc3[nH]c(N4c5ccccc5C(C...,22.699972,-0.078998,-13.192693,0.241512,-13.762622,0.751285,4.924800,1.448174,11.250433,1,7.136863,3.761683,9.007697,68.967000,1856.710784,5.386224
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
25281,Cn1cnc2ccc([SH]3C=CC=C3C=C(C#N)C#N)cc21,14.437825,-0.129878,-9.221304,0.040000,13.345314,0.959224,3.347107,1.266625,7.050781,4,19.930692,2.354344,5.318076,6.076020,871.118229,0.000000
25282,COc1cc(OC)cc(-c2cncc(-c3cccnc3)c2)c1,13.436442,-0.062498,-4.562327,0.041220,-29.750465,0.776395,4.308390,1.079007,8.203125,2,0.000000,2.906941,6.263008,6.066367,750.800749,0.000000
25283,N#CC(C#N)=C1C(=Cc2ccc(-c3ccccc3C(=O)C(F)(F)F)c...,20.974127,-0.061492,-6.395201,0.279771,-27.746302,0.526908,5.796932,1.445367,11.588477,2,18.792242,3.915485,8.648464,72.796405,1465.735283,22.253806
25284,CNc1cc(-c2ccc3c(c2)C(=O)C=CC3=O)cc(-c2c(F)c(F)...,24.199465,-0.080355,-11.249641,0.342822,-56.308941,0.543319,5.997693,1.380464,13.013291,0,2.837915,4.174679,9.881692,12.132734,1961.721603,16.814289


In [3]:
torch.manual_seed(0)

smiles = df["SMILES"]
featurizer = dc.feat.MolGraphConvFeaturizer(use_edges=True)
X = featurizer.featurize(smiles)

In [4]:
torch.manual_seed(0)

y_abcgg = df["ABCGG"]
data_train, data_temp, target_train_abcgg, target_temp_abcgg = train_test_split(X, y_abcgg, train_size=0.7, random_state=0)
data_val, data_test, target_val_abcgg, target_test_abcgg = train_test_split(data_temp, target_temp_abcgg, train_size=0.5, random_state=0)

target_train_abcgg = torch.tensor(target_train_abcgg.values, dtype=torch.float32)
target_val_abcgg = torch.tensor(target_val_abcgg.values, dtype=torch.float32)
target_test_abcgg = torch.tensor(target_test_abcgg.values, dtype=torch.float32)

data_train_list = []
for graph_data in data_train:
    node_features = torch.tensor(graph_data.node_features, dtype=torch.float32)
    edge_index = torch.tensor(graph_data.edge_index, dtype=torch.long)
    edge_features = torch.tensor(graph_data.edge_features, dtype=torch.float32)
    data = Data(x=node_features, edge_index=edge_index, edge_attr=edge_features)
    data_train_list.append(data)
    
data_val_list = []
for graph_data in data_val:
    node_features = torch.tensor(graph_data.node_features, dtype=torch.float32)
    edge_index = torch.tensor(graph_data.edge_index, dtype=torch.long)
    edge_features = torch.tensor(graph_data.edge_features, dtype=torch.float32)
    data = Data(x=node_features, edge_index=edge_index, edge_attr=edge_features)
    data_val_list.append(data)

data_test_list = []
for graph_data in data_test:
    node_features = torch.tensor(graph_data.node_features, dtype=torch.float32)
    edge_index = torch.tensor(graph_data.edge_index, dtype=torch.long)
    edge_features = torch.tensor(graph_data.edge_features, dtype=torch.float32)
    data = Data(x=node_features, edge_index=edge_index, edge_attr=edge_features)
    data_test_list.append(data)

In [5]:
torch.manual_seed(0)

with open('data_Random/data_train_list.pkl', 'wb') as f:
    pickle.dump(data_train_list, f)
with open('data_Random/data_val_list.pkl', 'wb') as f:
    pickle.dump(data_val_list, f)
with open('data_Random/data_test_list.pkl', 'wb') as f:
    pickle.dump(data_test_list, f)
with open('data_Random/target_train_abcgg.pkl', 'wb') as f:
    pickle.dump(target_train_abcgg, f)
with open('data_Random/target_val_abcgg.pkl', 'wb') as f:
    pickle.dump(target_val_abcgg, f)
with open('data_Random/target_test_abcgg.pkl', 'wb') as f:
    pickle.dump(target_test_abcgg, f)

In [6]:
torch.manual_seed(0)

y_aatsc3d = df["AATSC3d"]
data_train, data_temp, target_train_aatsc3d, target_temp_aatsc3d = train_test_split(X, y_aatsc3d, train_size=0.7, random_state=0)
data_val, data_test, target_val_aatsc3d, target_test_aatsc3d = train_test_split(data_temp, target_temp_aatsc3d, train_size=0.5, random_state=0)

target_train_aatsc3d = torch.tensor(target_train_aatsc3d.values, dtype=torch.float32)
target_val_aatsc3d = torch.tensor(target_val_aatsc3d.values, dtype=torch.float32)
target_test_aatsc3d = torch.tensor(target_test_aatsc3d.values, dtype=torch.float32)

with open('data_Random/target_train_aatsc3d.pkl', 'wb') as f:
    pickle.dump(target_train_aatsc3d, f)
with open('data_Random/target_val_aatsc3d.pkl', 'wb') as f:
    pickle.dump(target_val_aatsc3d, f)
with open('data_Random/target_test_aatsc3d.pkl', 'wb') as f:
    pickle.dump(target_test_aatsc3d, f)

In [7]:
torch.manual_seed(0)

y_atsc3d = df["ATSC3d"]
data_train, data_temp, target_train_atsc3d, target_temp_atsc3d = train_test_split(X, y_atsc3d, train_size=0.7, random_state=0)
data_val, data_test, target_val_atsc3d, target_test_atsc3d = train_test_split(data_temp, target_temp_atsc3d, train_size=0.5, random_state=0)

target_train_atsc3d = torch.tensor(target_train_atsc3d.values, dtype=torch.float32)
target_val_atsc3d = torch.tensor(target_val_atsc3d.values, dtype=torch.float32)
target_test_atsc3d = torch.tensor(target_test_atsc3d.values, dtype=torch.float32)

with open('data_Random/target_train_atsc3d.pkl', 'wb') as f:
    pickle.dump(target_train_atsc3d, f)
with open('data_Random/target_val_atsc3d.pkl', 'wb') as f:
    pickle.dump(target_val_atsc3d, f)
with open('data_Random/target_test_atsc3d.pkl', 'wb') as f:
    pickle.dump(target_test_atsc3d, f)

In [8]:
torch.manual_seed(0)

y_ggi10 = df["GGI10"]
data_train, data_temp, target_train_ggi10, target_temp_ggi10 = train_test_split(X, y_ggi10, train_size=0.7, random_state=0)
data_val, data_test, target_val_ggi10, target_test_ggi10 = train_test_split(data_temp, target_temp_ggi10, train_size=0.5, random_state=0)

target_train_ggi10 = torch.tensor(target_train_ggi10.values, dtype=torch.float32)
target_val_ggi10 = torch.tensor(target_val_ggi10.values, dtype=torch.float32)
target_test_ggi10 = torch.tensor(target_test_ggi10.values, dtype=torch.float32)

with open('data_Random/target_train_ggi10.pkl', 'wb') as f:
    pickle.dump(target_train_ggi10, f)
with open('data_Random/target_val_ggi10.pkl', 'wb') as f:
    pickle.dump(target_val_ggi10, f)
with open('data_Random/target_test_ggi10.pkl', 'wb') as f:
    pickle.dump(target_test_ggi10, f)

In [9]:
torch.manual_seed(0)

y_atsc4i = df["ATSC4i"]
data_train, data_temp, target_train_atsc4i, target_temp_atsc4i = train_test_split(X, y_atsc4i, train_size=0.7, random_state=0)
data_val, data_test, target_val_atsc4i, target_test_atsc4i = train_test_split(data_temp, target_temp_atsc4i, train_size=0.5, random_state=0)

target_train_atsc4i = torch.tensor(target_train_atsc4i.values, dtype=torch.float32)
target_val_atsc4i = torch.tensor(target_val_atsc4i.values, dtype=torch.float32)
target_test_atsc4i = torch.tensor(target_test_atsc4i.values, dtype=torch.float32)

with open('data_Random/target_train_atsc4i.pkl', 'wb') as f:
    pickle.dump(target_train_atsc4i, f)
with open('data_Random/target_val_atsc4i.pkl', 'wb') as f:
    pickle.dump(target_val_atsc4i, f)
with open('data_Random/target_test_atsc4i.pkl', 'wb') as f:
    pickle.dump(target_test_atsc4i, f)

In [10]:
torch.manual_seed(0)

y_bcutp1l = df["BCUTp-1l"]
data_train, data_temp, target_train_bcutp1l, target_temp_bcutp1l = train_test_split(X, y_bcutp1l, train_size=0.7, random_state=0)
data_val, data_test, target_val_bcutp1l, target_test_bcutp1l = train_test_split(data_temp, target_temp_bcutp1l, train_size=0.5, random_state=0)

target_train_bcutp1l = torch.tensor(target_train_bcutp1l.values, dtype=torch.float32)
target_val_bcutp1l = torch.tensor(target_val_bcutp1l.values, dtype=torch.float32)
target_test_bcutp1l = torch.tensor(target_test_bcutp1l.values, dtype=torch.float32)

with open('data_Random/target_train_bcutp1l.pkl', 'wb') as f:
    pickle.dump(target_train_bcutp1l, f)
with open('data_Random/target_val_bcutp1l.pkl', 'wb') as f:
    pickle.dump(target_val_bcutp1l, f)
with open('data_Random/target_test_bcutp1l.pkl', 'wb') as f:
    pickle.dump(target_test_bcutp1l, f)

In [11]:
torch.manual_seed(0)

y_kier3 = df["Kier3"]
data_train, data_temp, target_train_kier3, target_temp_kier3 = train_test_split(X, y_kier3, train_size=0.7, random_state=0)
data_val, data_test, target_val_kier3, target_test_kier3 = train_test_split(data_temp, target_temp_kier3, train_size=0.5, random_state=0)

target_train_kier3 = torch.tensor(target_train_kier3.values, dtype=torch.float32)
target_val_kier3 = torch.tensor(target_val_kier3.values, dtype=torch.float32)
target_test_kier3 = torch.tensor(target_test_kier3.values, dtype=torch.float32)

with open('data_Random/target_train_kier3.pkl', 'wb') as f:
    pickle.dump(target_train_kier3, f)
with open('data_Random/target_val_kier3.pkl', 'wb') as f:
    pickle.dump(target_val_kier3, f)
with open('data_Random/target_test_kier3.pkl', 'wb') as f:
    pickle.dump(target_test_kier3, f)

In [12]:
torch.manual_seed(0)

y_aats8p = df["AATS8p"]
data_train, data_temp, target_train_aats8p, target_temp_aats8p = train_test_split(X, y_aats8p, train_size=0.7, random_state=0)
data_val, data_test, target_val_aats8p, target_test_aats8p = train_test_split(data_temp, target_temp_aats8p, train_size=0.5, random_state=0)

target_train_aats8p = torch.tensor(target_train_aats8p.values, dtype=torch.float32)
target_val_aats8p = torch.tensor(target_val_aats8p.values, dtype=torch.float32)
target_test_aats8p = torch.tensor(target_test_aats8p.values, dtype=torch.float32)

with open('data_Random/target_train_aats8p.pkl', 'wb') as f:
    pickle.dump(target_train_aats8p, f)
with open('data_Random/target_val_aats8p.pkl', 'wb') as f:
    pickle.dump(target_val_aats8p, f)
with open('data_Random/target_test_aats8p.pkl', 'wb') as f:
    pickle.dump(target_test_aats8p, f)

In [13]:
torch.manual_seed(0)

y_kier2 = df["Kier2"]
data_train, data_temp, target_train_kier2, target_temp_kier2 = train_test_split(X, y_kier2, train_size=0.7, random_state=0)
data_val, data_test, target_val_kier2, target_test_kier2 = train_test_split(data_temp, target_temp_kier2, train_size=0.5, random_state=0)

target_train_kier2 = torch.tensor(target_train_kier2.values, dtype=torch.float32)
target_val_kier2 = torch.tensor(target_val_kier2.values, dtype=torch.float32)
target_test_kier2 = torch.tensor(target_test_kier2.values, dtype=torch.float32)

with open('data_Random/target_train_kier2.pkl', 'wb') as f:
    pickle.dump(target_train_kier2, f)
with open('data_Random/target_val_kier2.pkl', 'wb') as f:
    pickle.dump(target_val_kier2, f)
with open('data_Random/target_test_kier2.pkl', 'wb') as f:
    pickle.dump(target_test_kier2, f)

In [14]:
torch.manual_seed(0)

y_frnh0 = df["fr_NH0"]
data_train, data_temp, target_train_frnh0, target_temp_frnh0 = train_test_split(X, y_frnh0, train_size=0.7, random_state=0)
data_val, data_test, target_val_frnh0, target_test_frnh0 = train_test_split(data_temp, target_temp_frnh0, train_size=0.5, random_state=0)

target_train_frnh0 = torch.tensor(target_train_frnh0.values, dtype=torch.float32)
target_val_frnh0 = torch.tensor(target_val_frnh0.values, dtype=torch.float32)
target_test_frnh0 = torch.tensor(target_test_frnh0.values, dtype=torch.float32)

with open('data_Random/target_train_frnh0.pkl', 'wb') as f:
    pickle.dump(target_train_frnh0, f)
with open('data_Random/target_val_frnh0.pkl', 'wb') as f:
    pickle.dump(target_val_frnh0, f)
with open('data_Random/target_test_frnh0.pkl', 'wb') as f:
    pickle.dump(target_test_frnh0, f)

In [15]:
torch.manual_seed(0)

y_vsaestate3 = df["VSA_EState3"]
data_train, data_temp, target_train_vsaestate3, target_temp_vsaestate3 = train_test_split(X, y_vsaestate3, train_size=0.7, random_state=0)
data_val, data_test, target_val_vsaestate3, target_test_vsaestate3 = train_test_split(data_temp, target_temp_vsaestate3, train_size=0.5, random_state=0)

target_train_vsaestate3 = torch.tensor(target_train_vsaestate3.values, dtype=torch.float32)
target_val_vsaestate3 = torch.tensor(target_val_vsaestate3.values, dtype=torch.float32)
target_test_vsaestate3 = torch.tensor(target_test_vsaestate3.values, dtype=torch.float32)

with open('data_Random/target_train_vsaestate3.pkl', 'wb') as f:
    pickle.dump(target_train_vsaestate3, f)
with open('data_Random/target_val_vsaestate3.pkl', 'wb') as f:
    pickle.dump(target_val_vsaestate3, f)
with open('data_Random/target_test_vsaestate3.pkl', 'wb') as f:
    pickle.dump(target_test_vsaestate3, f)

In [16]:
torch.manual_seed(0)

y_kappa3 = df["Kappa3"]
data_train, data_temp, target_train_kappa3, target_temp_kappa3 = train_test_split(X, y_kappa3, train_size=0.7, random_state=0)
data_val, data_test, target_val_kappa3, target_test_kappa3 = train_test_split(data_temp, target_temp_kappa3, train_size=0.5, random_state=0)

target_train_kappa3 = torch.tensor(target_train_kappa3.values, dtype=torch.float32)
target_val_kappa3 = torch.tensor(target_val_kappa3.values, dtype=torch.float32)
target_test_kappa3 = torch.tensor(target_test_kappa3.values, dtype=torch.float32)

with open('data_Random/target_train_kappa3.pkl', 'wb') as f:
    pickle.dump(target_train_kappa3, f)
with open('data_Random/target_val_kappa3.pkl', 'wb') as f:
    pickle.dump(target_val_kappa3, f)
with open('data_Random/target_test_kappa3.pkl', 'wb') as f:
    pickle.dump(target_test_kappa3, f)

In [17]:
torch.manual_seed(0)

y_kappa2 = df["Kappa2"]
data_train, data_temp, target_train_kappa2, target_temp_kappa2 = train_test_split(X, y_kappa2, train_size=0.7, random_state=0)
data_val, data_test, target_val_kappa2, target_test_kappa2 = train_test_split(data_temp, target_temp_kappa2, train_size=0.5, random_state=0)

target_train_kappa2 = torch.tensor(target_train_kappa2.values, dtype=torch.float32)
target_val_kappa2 = torch.tensor(target_val_kappa2.values, dtype=torch.float32)
target_test_kappa2 = torch.tensor(target_test_kappa2.values, dtype=torch.float32)

with open('data_Random/target_train_kappa2.pkl', 'wb') as f:
    pickle.dump(target_train_kappa2, f)
with open('data_Random/target_val_kappa2.pkl', 'wb') as f:
    pickle.dump(target_val_kappa2, f)
with open('data_Random/target_test_kappa2.pkl', 'wb') as f:
    pickle.dump(target_test_kappa2, f)

In [18]:
torch.manual_seed(0)

y_peoevsa6 = df["PEOE_VSA6"]
data_train, data_temp, target_train_peoevsa6, target_temp_peoevsa6 = train_test_split(X, y_peoevsa6, train_size=0.7, random_state=0)
data_val, data_test, target_val_peoevsa6, target_test_peoevsa6 = train_test_split(data_temp, target_temp_peoevsa6, train_size=0.5, random_state=0)

target_train_peoevsa6 = torch.tensor(target_train_peoevsa6.values, dtype=torch.float32)
target_val_peoevsa6 = torch.tensor(target_val_peoevsa6.values, dtype=torch.float32)
target_test_peoevsa6 = torch.tensor(target_test_peoevsa6.values, dtype=torch.float32)

with open('data_Random/target_train_peoevsa6.pkl', 'wb') as f:
    pickle.dump(target_train_peoevsa6, f)
with open('data_Random/target_val_peoevsa6.pkl', 'wb') as f:
    pickle.dump(target_val_peoevsa6, f)
with open('data_Random/target_test_peoevsa6.pkl', 'wb') as f:
    pickle.dump(target_test_peoevsa6, f)

In [19]:
torch.manual_seed(0)

y_bertzct = df["BertzCT"]
data_train, data_temp, target_train_bertzct, target_temp_bertzct = train_test_split(X, y_bertzct, train_size=0.7, random_state=0)
data_val, data_test, target_val_bertzct, target_test_bertzct = train_test_split(data_temp, target_temp_bertzct, train_size=0.5, random_state=0)

target_train_bertzct = torch.tensor(target_train_bertzct.values, dtype=torch.float32)
target_val_bertzct = torch.tensor(target_val_bertzct.values, dtype=torch.float32)
target_test_bertzct = torch.tensor(target_test_bertzct.values, dtype=torch.float32)

with open('data_Random/target_train_bertzct.pkl', 'wb') as f:
    pickle.dump(target_train_bertzct, f)
with open('data_Random/target_val_bertzct.pkl', 'wb') as f:
    pickle.dump(target_val_bertzct, f)
with open('data_Random/target_test_bertzct.pkl', 'wb') as f:
    pickle.dump(target_test_bertzct, f)

In [20]:
torch.manual_seed(0)

y_estatevsa3 = df["EState_VSA3"]
data_train, data_temp, target_train_estatevsa3, target_temp_estatevsa3 = train_test_split(X, y_estatevsa3, train_size=0.7, random_state=0)
data_val, data_test, target_val_estatevsa3, target_test_estatevsa3 = train_test_split(data_temp, target_temp_estatevsa3, train_size=0.5, random_state=0)

target_train_estatevsa3 = torch.tensor(target_train_estatevsa3.values, dtype=torch.float32)
target_val_estatevsa3 = torch.tensor(target_val_estatevsa3.values, dtype=torch.float32)
target_test_estatevsa3 = torch.tensor(target_test_estatevsa3.values, dtype=torch.float32)

with open('data_Random/target_train_estatevsa3.pkl', 'wb') as f:
    pickle.dump(target_train_estatevsa3, f)
with open('data_Random/target_val_estatevsa3.pkl', 'wb') as f:
    pickle.dump(target_val_estatevsa3, f)
with open('data_Random/target_test_estatevsa3.pkl', 'wb') as f:
    pickle.dump(target_test_estatevsa3, f)